# Create Zarr dataset from the GOES-17 raster files
This notebook creates a zarr dataset from all the files stored in the directory `'/storage/GOES/orthorectified/Fog2022_withtime/'`. 

The zarr dataset is stored at the path `'/storage/GOES/orthorectified/Fog2022_withtime.zarr'`. 

The saved zarr dataset is optimized for access along the time-index, i.e. it allows us to grab time series for an individual pixel very quickly. 

We also utilized Dask, a Python library for parallel computing, to divide our large dataset into many smaller pieces, called chunks, each chunk small enough to load into memory. We used Dask every time we passed in an argument for “chunks” or called the “rechunk” method.

In [1]:
import os
import glob
import shutil
import xarray as xr
import zarr
from dask.distributed import Client, LocalCluster

In [2]:
fixed_image_folder = '/storage/GOES/orthorectified/Fog2022_withtime/'
zarr_output_path = '/storage/GOES/orthorectified/Fog2022_withtime.zarr'
# tmp_zarr_output_path = '/storage/GOES/orthorectified/Fog2022_withtime_tmp.zarr'

In [3]:
# Create a Dask cluster so we can watch the dask dashboard
# If this cell is not run, how many computer cores will be used?
workers = 6
ip_addres = 'http://j-lundquist-3.ce.washington.edu'
port=':8787'
threads = 2
cluster = LocalCluster(n_workers=workers, threads_per_worker=threads, dashboard_address=port)
client = Client(cluster)

### Sort the raster files chronologically

In [3]:
# Because the files in `'/storage/GOES/orthorectified/Fog2022_withtime/'` may not be in a chronological order, we sort them so that the timeseries data we are creating will be in a chronological order.

def get_start_date_from_G17_filename(s):
    return s.split('_G17_s')[1].split('_')[0]

nc_files = sorted(
    glob.glob(os.path.join(fixed_image_folder, '*.nc')),
    key=get_start_date_from_G17_filename
)

In [4]:
# The first files should be from May 1st which is the 121st day of the year, indicated by the string "s2022121_"
nc_files[:3]

['/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20221210001177_e20221210003550_c20221210007095_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20221210006177_e20221210008550_c20221210012545_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20221210011177_e20221210013550_c20221210016212_o.nc']

In [5]:
# The last files should be from September 30th which is the 273rd day of the year, indicated by the string "s2022273_"
nc_files[-3:]

['/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20222732341177_e20222732343550_c20222732346022_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20222732346177_e20222732348549_c20222732351218_o.nc',
 '/storage/GOES/orthorectified/Fog2022_withtime/OR_ABI-L2-ACHAC-M6_G17_s20222732351177_e20222732353549_c20222732356295_o.nc']

### Combine raster files into one dataset
Open by chunksename variable, change time zone.

In [6]:
# Open all the raster files as a single dataset (combining them together)
# Why did we choose chunks = 500? 100MB?
# https://docs.xarray.dev/en/stable/user-guide/dask.html#optimization-tips
ds = xr.open_mfdataset(nc_files, chunks={'time': 500})

In [7]:
# Rename HT as a more indicative name: Height
ds = ds.rename({'HT': 'Height'})

In [8]:
# Function to change timezone of the xarray dataset
def modify_xarray_timezone(ds, source_tz, target_tz):
    """Modify the timezone of an xr.Dataset. The dataset should have a coordinate and dimension 'time'.
    The returned xr.Dataset object will have the original 'time' coordinate/dimension overwritten.

    Args:
        ds (xr.Dataset): xarray Dataset object to have its time coordinate/dimension converted.
        source_tz (_type_): A pytz timezone object specifying the timezone the data is already in. 
                For example, `pytz.UTC`.
        target_tz (_type_): A pytz timezone object specifying the timezone the data is to be 
                converted to. For example, `pytz.timezone('US/Mountain')`.

    Returns:
        xr.Dataset: xarray Dataset with the time coordinate/dimension overwritten with the modified 
                timestamps.
    """
    ds = ds.copy()
    time_utc = ds['time'].to_index().tz_localize(source_tz)
    tz_corrected = time_utc.tz_convert(target_tz).tz_localize(None)
    local_da=xr.DataArray.from_series(tz_corrected)
    ds.coords.update({f'time ({target_tz})': tz_corrected})
    ds.coords.update({f'time ({source_tz})': ds['time'].to_index()})
    ds = ds.assign_coords({
        'time': ds[f'time ({target_tz})'].values
    })
    return ds

In [9]:
# Change from UTC time to US/Pacific timezone which the area of interest, Washington state, is in
ds = modify_xarray_timezone(ds, 'UTC', 'US/Pacific')

### Rechunk along the time dimension

In [10]:
# Dask's rechunk documentation: https://docs.dask.org/en/stable/generated/dask.array.rechunk.html

# 0:-1 specifies that we want the dataset to be chunked along the 0th dimension -- the time dimension, which means that each chunk will have all 40 thousand values in time dimension
# 1:'auto', 2:'auto' and balance=True specifies that dask can freely rechunk along the latitude and longitude dimensions to attain blocks that have a uniform size
ds['Height'].data.rechunk(
    {0:-1, 1:'auto', 2:'auto'}, 
    block_size_limit=1e8, 
    balance=True
)

# Assign the dimensions of a chunk to variables to use for encoding afterwards
t,y,x = ds['Height'].data.chunks[0][0], ds['Height'].data.chunks[1][0], ds['Height'].data.chunks[2][0]

In [11]:
ds['Height'].data

dask.array<concatenate, shape=(40785, 284, 284), dtype=float32, chunksize=(1, 284, 284), chunktype=numpy.ndarray>

In [12]:
# Create an output zarr file and write these chunks to disk
shutil.rmtree(zarr_output_path, ignore_errors=False)

In [13]:
ds['Height'].encoding = {'chunks': (t, y, x)}

ds.to_zarr(zarr_output_path)

/tmp/ipykernel_4829/2775166651.py:3: SerializationWarning: coordinate 'time (US/Pacific)' has a space in its name, which means it cannot be marked as a coordinate on disk and will be saved as a data variable instead
  ds.to_zarr(zarr_output_path)
/tmp/ipykernel_4829/2775166651.py:3: SerializationWarning: coordinate 'time (UTC)' has a space in its name, which means it cannot be marked as a coordinate on disk and will be saved as a data variable instead
  ds.to_zarr(zarr_output_path)


In [14]:
# Display 
source_group = zarr.open(zarr_output_path)
source_array = source_group['Height']
print(source_group.tree())
print(source_array.info)
del source_group
del source_array

/
 ├── Height (40785, 284, 284) float32
 ├── latitude (284,) float64
 ├── longitude (284,) float64
 ├── spatial_ref (40785,) int64
 ├── time (40785,) int64
 ├── time (US
 │   └── Pacific) (40785,) int64
 └── time (UTC) (40785,) int64
Name               : /Height
Type               : zarr.core.Array
Data type          : float32
Shape              : (40785, 284, 284)
Chunk shape        : (1, 284, 284)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : zarr.storage.DirectoryStore
No. bytes          : 13158219840 (12.3G)
No. bytes stored   : 1938406915 (1.8G)
Storage ratio      : 6.8
Chunks initialized : 40785/40785

